In [1]:
import os, sys, time, subprocess
import numpy as np
import pandas as pd
from PIL import Image
from keras import models
from keras import backend as K
from keras.preprocessing import image

sys.path.insert(0, os.path.abspath('../utils/'))
import util as utils

Using TensorFlow backend.


In [2]:
# Some memory clean-up
K.clear_session()

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [4]:
def convert_bytecodes_to_images(bytecodes_files, bytecodes_dir, test_case_images_dir):
    if not os.path.exists(test_case_images_dir):
        os.mkdir(test_case_images_dir)
    
    image_set = []

    start_convert_images = time.monotonic()

    for filename in bytecodes_files:
        image_set.append([ utils.byte2img(os.path.join(bytecodes_dir, filename+'.bytes')), filename ])

    end_convert_images = time.monotonic()
    
    print('Time taken to convert bytecodes to images: ', (end_convert_images - start_convert_images))
    print('Number of images converted: ', len(image_set))
    print()
    
    for image in image_set:
        utils.resize_from_img(image[0], os.path.join(test_case_images_dir, image[1]+'.png'))

In [5]:
def load_images_for_prediction(bytecodes_files, test_case_images_dir):
    test_case_images = [ utils.load_image_as_np(os.path.join(test_case_images_dir, (filename+'.png'))) for filename in bytecodes_files ]
    test_case_images_set = np.asarray(test_case_images)
    test_case_images_set = test_case_images_set.astype('float32') / 255
    return test_case_images_set

In [6]:
def load_model(model_file):
    return models.load_model(model_file)

In [7]:
subprocess.call('./convert_exe_to_bytes.sh')

0

In [8]:
model_file = '../final_trained_model.h5'
bytecodes_dir = './test_cases/'
test_case_images_dir = os.path.abspath('./test_case_images/')

In [9]:
bytecodes_files = [ os.path.splitext(os.path.basename(file))[0] for file in sorted(os.listdir(bytecodes_dir)) ]

In [10]:
convert_bytecodes_to_images(bytecodes_files, bytecodes_dir, test_case_images_dir)
test_case_images_set = load_images_for_prediction(bytecodes_files, test_case_images_dir)
model = load_model(model_file)

Time taken to convert bytecodes to images:  3.6893702630000007
Number of images converted:  2



/home/student/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image/utils.py:98: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [11]:
predicted_labels = model.predict_classes(test_case_images_set, batch_size=128)

In [12]:
label_mapping = { label: family for label, family in pd.read_csv('./label_mapping.csv').get_values() }

In [13]:
predicted_labels_mapped = [ label_mapping.get(label, label) for label in predicted_labels ]

In [14]:
final_predicted_labels = [ [bytecodes_files[i], label] for i, label in enumerate(predicted_labels_mapped) ]

In [15]:
for final_predicted_label in final_predicted_labels:
    print('Filename: ', (final_predicted_label[0]+'.bytes'))
    print('Predicted malware family: ', (final_predicted_label[1]))
    print()

Filename:  WannaCryVirus.bytes
Predicted malware family:  Gatak

Filename:  chrome.bytes
Predicted malware family:  Ramnit

